## Variation Quantum Eigensolver

It's a hybrid quantum-classical algorithm used to find the ground state energy of a quantum system, represented by Hamiltonian $H$ such that
\begin{equation}
H \psi_0 = E_0 \psi_0 
\end{equation}
. For most realistic systems, solving this equation exactly is impossible. Computing this exactly for large systems is classically intractable because the Hilbert space grows exponentially with system size. 

VQE is particularly well-suited for noisy intermediate-scale quantum (NISQ) devices, as it minimizes the number of required qubits and circuit depths compared to algorithms like Quantum Phase Estimation (QPE).

---

### Variational Principle

The foundation of VQE lies in the variational principle of quantum mechanics:

> For any normalized trial wavefunction $ |\psi(\boldsymbol{\theta})\rangle $,  
> the expectation value of the Hamiltonian provides an upper bound to the ground state energy $ E_0 $:
>
> 
> $E(\boldsymbol{\theta}) = $
> $\langle \psi(\boldsymbol{\theta}) | H | \psi(\boldsymbol{\theta}) \rangle $
> $\geq E_0$
> 

Hence, if we can minimize $ E(\boldsymbol{\theta}) $ over a suitable set of parameterized states $ |\psi(\boldsymbol{\theta})\rangle $,  
the minimum value we obtain will approximate the ground state energy.

---

### Parameterized Quantum States (Ansatz)

In VQE, the trial state $ |\psi(\boldsymbol{\theta})\rangle$ is prepared by a parameterized quantum circuit, also known as an ansatz:

$
|\psi(\boldsymbol{\theta})\rangle = U(\boldsymbol{\theta}) |0\rangle
$

Here:
- $ U(\boldsymbol{\theta}) $ is a unitary circuit composed of rotations and entangling gates,  
- $ \boldsymbol{\theta} = (\theta_1, \theta_2, \ldots, \theta_n) $ are tunable classical parameters.

The choice of ansatz is crucial:
- It must be expressive enough to capture the true ground state,
- But also efficient to implement on real quantum hardware.

Common examples:
- Hardware-efficient ansatz (e.g., alternating RY rotations and CZ entanglers),
- Chemistry-inspired ansatz (e.g., UCCSD — Unitary Coupled Cluster with Single and Double excitations).

---

### Hybrid Quantum–Classical Loop

VQE minimizes the expectation value

$
E(\boldsymbol{\theta}) = 
\sum_i c_i \langle \psi(\boldsymbol{\theta}) | P_i | \psi(\boldsymbol{\theta}) \rangle
$

where the Hamiltonian has been decomposed into Pauli strings:

$
H = \sum_i c_i P_i, \quad P_i \in \{ I, X, Y, Z \}^{\otimes n}
$

The algorithm proceeds iteratively:

1. Quantum step:  
   The quantum processor prepares $ |\psi(\boldsymbol{\theta})\rangle $ and measures expectation values  
   $ \langle P_i \rangle = \langle \psi(\boldsymbol{\theta}) | P_i | \psi(\boldsymbol{\theta}) \rangle $.

2. Classical step:
   A classical optimizer (e.g., COBYLA, SPSA, L-BFGS-B) updates the parameters  
   $ \boldsymbol{\theta} \to \boldsymbol{\theta}' $ to minimize the total energy $ E(\boldsymbol{\theta}) $.

3. Convergence:
   The process repeats until $ E(\boldsymbol{\theta})$  converges to a minimum, ideally close to the ground state energy $ E_0 $.

---

### Mathematical Formulation

At convergence:

$
\boldsymbol{\theta}^* = 
\arg \min_{\boldsymbol{\theta}} 
\langle \psi(\boldsymbol{\theta}) | H | \psi(\boldsymbol{\theta}) \rangle
$

and the corresponding minimum energy:

$
E_{\text{min}} = E(\boldsymbol{\theta}^*) \approx E_0
$

The quality of the approximation depends on:
- The expressivity of the ansatz,
- The accuracy of the quantum measurements,
- The effectiveness of the classical optimizer.

---

### Advantages and Limitations

#### Advantages
- Requires shallow circuits, suitable for NISQ devices.
- Can handle a wide variety of Hamiltonians (e.g., molecular, spin, lattice models).
- Naturally integrates with quantum chemistry frameworks.

#### Limitations
- Barren plateaus: gradients vanish in large ansätze.
- Shot noise and hardware errors affect energy estimation.
- Convergence can depend heavily on the optimizer and initial parameters.


## Example: Ground state energy of $H_2$ molecule

The Hamiltonian for hydrogen molecule can be approximated by a two qubit Hamiltonian,
\begin{equation}
H = c_0 + c_1 Z \otimes I + c_2 I \otimes Z + c_3 Z \otimes Z + c_4 (X \otimes X + Y \otimes Y)
\end{equation}
where $c = [-1.052373245772859, 0.39793742484318045, -0.39793742484318045, -0.01128010425623538, 0.18093119978423156]$.

### Classical benchmarking using NumPy

In [ ]:
#Building the 2 qubit Hamiltonian for Hydrogen molecule
from qiskit.quantum_info import SparsePauliOp

coeffs = [-1.052373245772859, 0.39793742484318045, -0.39793742484318045, -0.01128010425623538, 0.18093119978423156];
H2 = (
    SparsePauliOp.from_list([("II", coeffs[0])]) +
    SparsePauliOp.from_list([("ZI", coeffs[1])]) +
    SparsePauliOp.from_list([("IZ", coeffs[2] )]) +
    SparsePauliOp.from_list([("ZZ", coeffs[3])]) +
    SparsePauliOp.from_list([("XX", coeffs[4])]) +
    SparsePauliOp.from_list([("YY", 0)])
)

#Using the NumPyMinimumEigensolver to find the minimum energy of H2
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=H2)
energy_ref = result.eigenvalue.real
print(f"Reference value: {energy_ref:.5f} Hartree")

### Running on quantum hardware using COBYLA

In [ ]:
#Ansztaz for the trial wave function
from qiskit.circuit.library import n_local

ansatz = n_local(2, rotation_blocks="ry", entanglement_blocks="cz",reps=1)

#To compute the expectation value
from qiskit.quantum_info import Statevector
import numpy as np

def expectation_value(params):
    bound_circuit = ansatz.assign_parameters(params)
    psi = Statevector.from_instruction(bound_circuit)
    return np.real(psi.expectation_value(H2))

#COBYLA optimizer to find the minimum energy
from qiskit_algorithms.optimizers import COBYLA

optimizer = COBYLA(maxiter=100, tol=1e-6)
result = optimizer.minimize(fun=expectation_value, x0=np.array([np.pi/4, np.pi/3,0,0]))    

print(f"Approximated ground state energy: {result.fun:.5f} Hartree" )

### Without backend noise using VQE

In [ ]:
from qiskit_aer.primitives import EstimatorV2 as AerEstimator
from qiskit_algorithms import VQE

noiseless_estimator = AerEstimator();

vqe = VQE(noiseless_estimator, ansatz, optimizer)
vqe_result = vqe.compute_minimum_eigenvalue(operator=H2)

print(f"Approximated ground state energy =  {vqe_result.eigenvalue.real:.5f} Hartree" )

### With backend noise

In [ ]:
# Creating a 5-qubit noisy backend

from qiskit_aer.noise import NoiseModel
from qiskit.providers.fake_provider import GenericBackendV2

coupling_map=[(0,1),(1,2),(2,3),(3,4)];
device = GenericBackendV2(num_qubits=5, coupling_map = coupling_map)
noise_model = NoiseModel.from_backend(device)

print("Noise model summary: ")
print(noise_model)

In [ ]:
#Create an Estimator primitive backed by the noisy model
noisy_estimator = AerEstimator(
    options={
        "default_precision": 1e-2,
        "backend_options": {
            "method": "density_matrix",
            "coupling_map": coupling_map,
            "noise_model": noise_model,
        },
    }
)
vqe.estimator = noisy_estimator

result_n = vqe.compute_minimum_eigenvalue(operator=H2)

print(f"Approximated ground state energy =  {result_n.eigenvalue.real:.5f} Hartree" )